In [ ]:
import numpy as np

import theano 
import theano.tensor as T
from theano.tensor.signal import pool
from theano.tensor.nnet import conv2d


import IPython
import matplotlib.pyplot as plt
import timeit

%pylab inline
pylab.rcParams['figure.figsize'] = (6, 4)

%matplotlib inline
plt.rcParams['figure.figsize'] = (6.0, 5.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# State Farm Distracted Drivers

<img src="../resources/drivers_statefarm.png" width="900">


- c0: safe driving
- c1: texting - right
- c2: talking on the phone - right
- c3: texting - left
- c4: talking on the phone - left
- c5: operating the radio
- c6: drinking
- c7: reaching behind
- c9: talking to passenger


## Les données 

79,726 photos à classer (ici on n'en charge que 200) 
22,424 photos d'entrainement (pour l'apprentissage et la validation)


In [ ]:
X_test = np.load('../data/state-farm/X_test_40.npy')

# Pour récupérer les id des photos, pour faire une soumission sur Kaggle
X_test_id = np.load('../data/state-farm/X_test_id_40.npy')
print(X_test.shape,X_test_id.shape)

In [ ]:
X_train = np.load('../data/state-farm/X_train_40.npy')
Y_train = np.load('../data/state-farm/y_train_40.npy')
print(X_train.shape, Y_train.shape)

In [ ]:
X_valid= np.load('../data/state-farm/X_valid_40.npy')
Y_valid = np.load('../data/state-farm/y_valid_40.npy')
print(X_valid.shape, Y_valid.shape)

In [ ]:
plt.imshow(X_valid[1234].reshape(40,40))


In [ ]:
print(Y_valid[1234])

In [ ]:
# On charge les données comme des variables partagées, pour le mini-batch SGD
X_train_shared =  T.cast(theano.shared(X_train),'float32')
y_train_shared = T.cast(theano.shared(Y_train),'int32')

X_valid_shared =  T.cast(theano.shared(X_valid),'float32')
y_valid_shared = T.cast(theano.shared(Y_valid),'int32')

X_test_shared =  T.cast(theano.shared(X_test),'float32')


## Construire le modèle 

In [ ]:
learning_rate=0.01
n_epochs=50
batch_size=200
L1_reg=0.00
L2_reg=0.0001

n_kern1 = ...
n_kern2 = ...

n_features_map_1 = ...
n_features_map_2 = ...

n_out_fc1 = ...
n_out_fc2 = ...
n_out_softmax = 10

n_train_batches = X_train.shape[0] // batch_size
n_valid_batches = X_valid.shape[0] // batch_size


rng = np.random.RandomState(1234)

In [ ]:
index = ...

X = ...
Y = ...

In [ ]:
# On commence par une première couche qui va changer les dimensions de la 
layer0_input = X.reshape((batch_size, 1, 40, 40))

In [ ]:
# Dimensions des données (#batch_size, 1 channel, 64x64)
image_shape_layer1=(batch_size, 1, 40, 40)

# dimension des paramètres: (#feature map en sortie, #feature map en entrée, #size_kernel, #size_kernel)
filter_shape_layer1=(... ,..., ...,...)

poolsize_layer1=(2, 2)



# Poids des filtres de la première couche de convolution
W_value = np.asarray( rng.uniform( size = .... ),
                        dtype=theano.config.floatX)

W_layer1 = theano.shared( value = W_value )



b_value = np.zeros( (...,), 
                   dtype=theano.config.floatX)

b_layer1 = theano.shared(value= b_value)



conv_out_layer1 = conv2d(
            input=...,
            filters=...,
            filter_shape=...,
            input_shape=...
        )

pooled_out_layer1 = pool.pool_2d(
            input=...,
            ds=...,
            ignore_border=True
        )


#On passe la sortie + le biais dans la fonction d'activation 
output_layer1 = T.nnet.relu( pooled_out_layer1 + b_layer1.dimshuffle('x', 0, 'x', 'x'))

In [ ]:
image_shape_layer2=(..., ..., ..., ...)
filter_shape_layer2=(..., ..., ..., ...)
poolsize_layer2=(2, 2)


W_value = np.asarray( rng.uniform( size = .... ),
                        dtype=theano.config.floatX)

W_layer2 = theano.shared( value = W_value )

b_value = np.zeros( (...,), 
                   dtype=theano.config.floatX)

b_layer2 = theano.shared(value= b_value)



conv_out_layer2 = conv2d(
            input=...,
            filters=...,
            filter_shape=...,
            input_shape=...
        )



pooled_out_layer2 = pool.pool_2d(
            input=...,
            ds=poolsize_layer2,
            ignore_border=True
        )


output_layer2 = T.nnet.relu(pooled_out_layer2 + b_layer2.dimshuffle('x', 0, 'x', 'x'))

In [ ]:
input_layer3 = output_layer2.flatten(2)

n_in_fc1 = n_features_map_2 * 5 * 5

W_layer3 = theano.shared(value=np.asarray(
                                    rng.uniform( low=-np.sqrt(6. / (n_in_fc1 + n_out_fc1)),
                                                 high=np.sqrt(6. / (n_in_fc1 + n_out_fc1)),
                                                size=(n_in_fc1, n_out_fc1)),
                                    dtype=theano.config.floatX),
                    name='W_layer3', borrow=True)

        
b_layer3 = theano.shared(value=np.zeros((n_out_fc1,), dtype=theano.config.floatX), name='b_layer3', borrow=True)

output_layer3 = T.nnet.relu(T.dot(input_layer3, W_layer3) + b_layer3)

In [ ]:
n_in_fc2 = n_out_fc1

W_layer4 = theano.shared(value=np.asarray(
                                    rng.uniform( low=-np.sqrt(6. / (n_in_fc2 + n_out_fc2)),
                                                 high=np.sqrt(6. / (n_in_fc2 + n_out_fc2)),
                                                size=(n_in_fc2, n_out_fc2)),
                                    dtype=theano.config.floatX),
                    name='W_layer4', borrow=True)

        
b_layer4 = theano.shared(value=np.zeros((n_out_fc2,), dtype=theano.config.floatX), name='b_layer4', borrow=True)

output_layer4 = T.nnet.relu(T.dot(output_layer3, W_layer4) + b_layer4)

In [ ]:
W_logReg = theano.shared(
            value=np.zeros(
                (n_out_fc2, n_out_softmax),
                dtype=theano.config.floatX
            ),
            name='W_logReg',
            borrow=True
        )
b_logReg = theano.shared(
            value=np.zeros(
                (n_out_softmax,),
                dtype=theano.config.floatX
            ),
            name='b_logReg',
            borrow=True
        )

In [ ]:
params = [W_layer1, b_layer1, W_layer2, b_layer2, W_layer3, b_layer3, W_layer4, b_layer4,W_logReg, b_logReg]

L1 = ...
L2 = ...



In [ ]:
p_y_given_x = ...

cost = ...

y_pred = ...

errors = ...

In [ ]:
validate_model = ...

In [ ]:
gparams = ...

updates = ...

train_model = ...

In [ ]:
test_model = theano.function(
        inputs=[],
        outputs=p_y_given_x, 
        givens={
            x: X_test_shared[:]
        }
    )

In [ ]:
best_validation_loss = np.inf
validation_frequency = 500

start_time = timeit.default_timer()
for epoch in range(n_epochs):
    for minibatch_index in range(n_train_batches):
        
        minibatch_avg_cost = train_model(minibatch_index)
            
        # iteration number
        iter = (epoch) * n_train_batches + minibatch_index

        if (iter + 1) % validation_frequency == 0:
            this_validation_loss = validate_model(X_valid,y_valid.astype('int32'))*100.

            pylab.plot([iter], [this_validation_loss], 'bo', label='validation_loss')
            pylab.plot([iter], [minibatch_avg_cost[1]], 'ro', label='training_loss')
            IPython.display.clear_output(wait=True)
            IPython.display.display(pylab.gcf())
        
            
            if this_validation_loss < best_validation_loss:
                best_validation_loss = this_validation_loss

end_time = timeit.default_timer()

print(( 'Entrainement fini : %f %%') % (best_validation_loss * 100.))
    
print('en %d secondes, avec %f epochs/sec' % ((end_time - start_time), 1. * epoch / (end_time - start_time)))

In [ ]:
predictions = test_model()